In [ ]:
import pyVoxelFEM, MeshFEM, visualization, benchmark, parallelism
import numpy as np
import matplotlib, matplotlib.pyplot as plt
from CoarseningLevelBenchmark import optMGLevel
import sys, os, psutil

parallelism.set_max_num_tbb_threads(psutil.cpu_count(logical=False))
parallelism.set_gradient_assembly_num_threads(min(psutil.cpu_count(logical=False), 8))

In [ ]:
DIM=2

BC_PATH = '../examples/bcs/cantilever_flexion_SE.bc'
domainCorners = [[0, 0], [2, 1]]
gridResolution = [384, 192]

maxVol = 0.33
constraints = [pyVoxelFEM.TotalVolumeConstraint(maxVol)]

# Set up the chain of density filters
sf = pyVoxelFEM.SmoothingFilter(3)
sf.type = sf.Type.Linear
pf = pyVoxelFEM.ProjectionFilter(1)
filters = [sf, pf] # Smoothing filter followed by projection

tps = pyVoxelFEM.TensorProductSimulator([1] * DIM, domainCorners, gridResolution)
tps.readMaterial('../examples/materials/B9Creator.material')
tps.applyDisplacementsAndLoadsFromFile(BC_PATH)

objective = pyVoxelFEM.MultigridComplianceObjective(tps.multigridSolver(optMGLevel(gridResolution, DIM)))
top = pyVoxelFEM.TopologyOptimizationProblem(tps, objective, constraints, filters)
top.setVars(pf.invert(maxVol) * np.ones(top.numVars()));

# BC visualization

In [ ]:
view = visualization.TPSViewer(tps)
view.show()

In [ ]:
max_iters=50
oco = pyVoxelFEM.OCOptimizer(top)
benchmark.reset()
for i in range(max_iters):
    view.update()
    oco.step()
# Continue optimization with a steeper projection filter
top.filterChain.filters[-1].beta = 8
tps.gamma = 5
for i in range(max_iters):
    view.update()
    oco.step()
benchmark.report()

In [ ]:
view.show()

In [ ]:
# Change colormap
view.densityColormap = matplotlib.cm.jet

In [ ]:
# Hide the voxels below a given threshold
view.densityThreshold = 0.5 

In [ ]:
# Disable thresholding
view.densityThreshold = None

In [ ]:
# Animate the displacement field in the voxel-based viewer
for t in np.linspace(0, 4 * np.pi, 200):
    view.applyDisplacements(top.objective.u(), 0.001 * np.sin(t))